#### Installing AutoGluon

In [ ]:
!pip install autogluon

##### unzip

In [3]:
!unzip /content/drive/MyDrive/전력사용량/open.zip

Archive:  /content/drive/MyDrive/전력사용량/open.zip
  inflating: building_info.csv       
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
import pandas as pd

In [5]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [8]:
#결측값을 0으로 채웁니다
train_df = train_df.fillna(0)

#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['month'] = train_df['일시'].apply(lambda x : int(x[4:6]))
train_df['day'] = train_df['일시'].apply(lambda x : int(x[6:8]))
train_df['time'] = train_df['일시'].apply(lambda x : int(x[9:11]))

# train_x = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
train_x = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)'])
train_y = train_df['전력소비량(kWh)']

test_df['month'] = test_df['일시'].apply(lambda x : int(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : int(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : int(x[9:11]))

test_x = test_df.drop(columns=['num_date_time', '일시'])

In [ ]:
# '전력소비량(kWh)'

In [13]:
train_data = TabularDataset(train_x)
test_data = TabularDataset(test_x)

save_path = 'AutogluonModels/mae_360000'
predictor = TabularPredictor(
    label='전력소비량(kWh)',
    path=save_path,
    eval_metric='mean_absolute_percentage_error'
    ).fit(
        train_data=train_data,
        presets='best_quality',
        time_limit=3600*1
        )

pred = predictor.predict(test_data)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/mae_360000/"
AutoGluon Version:  0.8.2
Python Version:     3.10.6
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   83.23 GB / 115.66 GB (72.0%)
Train Data Rows:    204000
Train Data Columns: 8
Label Column: 전력소비량(kWh)
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (25488.4, 0.0, 2451.03646, 2440.64886)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to prepro

In [14]:
pred

0        1689.140503
1        1841.205200
2        1645.944946
3        1197.158325
4        1278.095337
            ...     
16795    1252.642090
16796    1306.275513
16797    1237.789917
16798    1029.661377
16799     948.643982
Name: 전력소비량(kWh), Length: 16800, dtype: float32

In [15]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = pred
submission.to_csv('./baseline_submission.csv', index=False)